In [0]:
!pip install twitter_scraper

In [0]:
!pip install basilica

In [0]:
!wget https://storage.googleapis.com/basilica-public/cats_dogs_demo.tgz
!tar -xf cats_dogs_demo.tgz

In [0]:
from twitter_scraper import get_tweets
import basilica

In [0]:
conn = basilica.Connection('6d34b5f8-7239-219b-fd71-33fcfe9e24a8')

In [0]:
users = ['fchollet', 'kennethreitz', 'youngchingjui']

for user in users:
  tweet_list = []
  for tweet in get_tweets(user, pages=1):
    tweet_list.append(tweet['text'])
  tweet_embeddings = conn.embed_sentences(tweet_list, model='twitter')
  embedding_list = list(tweet_embeddings)

In [52]:
tweet_list = []
for tweet in get_tweets('fchollet', pages=1):
  tweet_list.append(tweet['text'])
#   print(tweet['text'])
len(tweet_list)

21

In [45]:
tweet_list = []
for tweet in get_tweets('kennethreitz', pages=1):
  tweet_list.append(tweet['text'])
#   print(tweet['text'])
len(tweet_list)

21

In [47]:
tweet_list = []
for tweet in get_tweets('youngchingjui', pages=1):
  tweet_list.append(tweet['text'])
  print(tweet['text'])
len(tweet_list)

I hate using spotlight to open the Photos app on Mac.

Spotlight search:
"p" - Photos
"ph" - Photo Booth
"pho" - Photos
"photo" = Photo Booth
"photos" = Photos

...and I can't delete Photo Booth either
That's my update for this week! If you're also self-employed and looking for a community to help keep you accountable, feel free to join in on this #WeeklyTwitterStandup. Just @ me with a day of the week you want to post, and I'll ping to remind you.
Also designing a website from scratch on #Webflow. Both design tools are great and fully functioning - but again, this is my first time, so it's a bit of a learning curve.
What I'm stuck on:
Designing a podcast app on #Sketch. I'm new to designing apps, and also new with Sketch. So it's taking a while to learn how to use Symbols (components), especially nested Symbols
What I will do:
Finish podcast design so I can start coding it with #ReactNative
Finish wedding website - optimized for both desktop and mobile
 https://twitter.com/youngchingj

20

In [0]:
embeddings = conn.embed_sentences(tweet_list, model='twitter')
embeddings = list(embeddings)

In [29]:
len(embeddings[2][:5])

5

In [30]:
embeddings[2][:5]

[-0.0305636, -0.295747, 1.36708, -0.922467, 1.0826]

In [43]:
len(embeddings)

21

# Basilica Image Embedding

In [9]:
import basilica
from six.moves import zip
import json
import os

EMB_DIR = '/tmp/basilica-embeddings/'
if not os.path.exists(EMB_DIR):
    os.mkdir(EMB_DIR)

IMG_DIR = 'cats_dogs_demo/images/'
API_KEY = '6d34b5f8-7239-219b-fd71-33fcfe9e24a8'
with basilica.Connection(API_KEY) as c:
    filenames = os.listdir(IMG_DIR)
    embeddings = c.embed_image_files(IMG_DIR + f for f in filenames)
    for filename, embedding in zip(filenames, embeddings):
        with open(EMB_DIR + filename + '.emb', 'w') as f:
            f.write(json.dumps(embedding))
            print(filename)

dog.897.jpg
cat.616.jpg
cat.480.jpg
cat.259.jpg
dog.1213.jpg
dog.1021.jpg
cat.298.jpg
dog.434.jpg
dog.469.jpg
cat.1487.jpg
cat.664.jpg
cat.817.jpg
dog.642.jpg
dog.196.jpg
cat.1004.jpg
dog.1224.jpg
dog.1303.jpg
cat.807.jpg
dog.685.jpg
dog.141.jpg
dog.1350.jpg
cat.5.jpg
dog.565.jpg
dog.1127.jpg
cat.853.jpg
cat.1040.jpg
cat.78.jpg
dog.403.jpg
cat.884.jpg
cat.1105.jpg
cat.586.jpg
cat.1173.jpg
cat.755.jpg
cat.318.jpg
dog.72.jpg
cat.1214.jpg
dog.1080.jpg
cat.329.jpg
cat.1237.jpg
cat.465.jpg
dog.61.jpg
cat.865.jpg
cat.234.jpg
dog.525.jpg
dog.1198.jpg
dog.1226.jpg
dog.1172.jpg
cat.938.jpg
cat.1160.jpg
dog.1156.jpg
dog.732.jpg
dog.221.jpg
dog.369.jpg
dog.787.jpg
cat.1014.jpg
dog.1394.jpg
dog.950.jpg
cat.634.jpg
dog.1033.jpg
dog.810.jpg
cat.924.jpg
dog.1177.jpg
cat.968.jpg
dog.68.jpg
dog.84.jpg
cat.379.jpg
cat.376.jpg
cat.1145.jpg
dog.1491.jpg
dog.715.jpg
dog.116.jpg
dog.129.jpg
cat.700.jpg
dog.568.jpg
cat.1251.jpg
dog.1338.jpg
dog.770.jpg
cat.111.jpg
dog.721.jpg
cat.106.jpg
cat.730.jpg
cat.1211

## Training a classifier

In [0]:
import json
import numpy as np
import os
import random
import re
import sklearn.linear_model
import sklearn.preprocessing
import time

In [0]:
EMB_DIR = '/tmp/basilica-embeddings/'
files = [f for f in os.listdir(EMB_DIR)]
random.shuffle(files)
train_size = int(len(files)*0.8)

x_train = np.zeros((train_size, 2048))
x_test = np.zeros((len(files)-train_size, 2048))
y_train = np.zeros(train_size, dtype=int)
y_test = np.zeros(len(files)-train_size, dtype=int)

for i in range(train_size):
    filename = files[i]
    with open(EMB_DIR + filename, 'r') as f:
        x_train[i] = json.load(f)
        y_train[i] = (0 if re.match('.*cat.*', filename) else 1)

for i in range(len(files) - train_size):
    filename = files[train_size+i]
    with open(EMB_DIR + filename, 'r') as f:
        x_test[i] = json.load(f)
        y_test[i] = (0 if re.match('.*cat.*', filename) else 1)

In [12]:
x_train = sklearn.preprocessing.normalize(x_train)
x_test = sklearn.preprocessing.normalize(x_test)
model = sklearn.linear_model.LogisticRegression()
model.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
print('Train accuracy: %.3f' % model.score(x_train, y_train))
print('Test accuracy: %.3f' % model.score(x_test, y_test))

Train accuracy: 0.989
Test accuracy: 0.988


## What sorts of mistakes are we making?

![alt text](https://www.basilica.ai/static/images/tutorial/dog.1004.jpg)![alt text](https://www.basilica.ai/static/images/tutorial/cat.991.jpg)![alt text](https://www.basilica.ai/static/images/tutorial/dog.412.jpg)

In [14]:
test_proba = model.predict_proba(x_test)
collected = zip(files[train_size:], y_test, test_proba)
probabilities = [(pred[y], f) for f, y, pred in collected]
probabilities.sort()
for prob, filename in probabilities[:3]:
    print('%s: %.2f' % (filename, prob))

cat.674.jpg.emb: 0.20
cat.720.jpg.emb: 0.38
cat.426.jpg.emb: 0.39
